In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn import metrics
import json
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
import collections
from itertools import groupby
import pickle
import re
import matplotlib.colors as mcolors
import warnings
import requests
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from functools import reduce
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

### Global Variables & Functions

#### Variables

In [3]:
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

colors = [color[4:] for color in mcolors.TABLEAU_COLORS.keys()]
colors.extend([color for color in mcolors.CSS4_COLORS.keys()])
colors.extend(
    [
        'whitish', 'bluish', 'reddish', 'greenish', 'backish', 'greyish',
        'backish', 'purplish', 'yellowish', 'orangish', 'brownish', 'pinkish'
    ]
)



traits_dict = {
    'Life Form':
    [
        'Tree', 'Shrub', 'Bush', 'Ficus', 'Strangler', 'Liana', 'Parasitic', 'Palm', 'Herbaceous'
    ],
    'Trunk':
    [
        'Trunk', 'Straight', 'Flared', 'Foothills', 'Silt', 'Aerial'
    ],
    'Root':
    [
        'Root', 'Straight', 'Flared', 'Foothills', 'Silt', 'Aerial'
    ],
    'Latex':
    [
        'Latex'
    ],
    'Phyllotaxis': # Leaf Position
    [
        'Phyllotaxis', 'Alternate', 'Whorled', 'Whorls', 'Opposite'
    ],
    'Leaf Composition':
    [
        'Palmate', 'Pinnate', 'Entire', 'Bi-pinnate'
    ],
    'Crown':
    [
        'Crown'
    ],
    'Stem':
    [
        'Stem', 'Circular', 'Square'
    ],
    'Bark':
    [
        'Bark'
    ],
    'Bark Colour':
    [
        'Bark'
    ],
    'Leaf Shape':
    [
        'Simple', 'Bifoliate', 'Trifoliate', 'Digitized', 'Paripinnate', 'Unipinnate', 'Imperipinnate', 
        'Alternate', 'Bipinnate', 'Pinnate', 'Elliptic', 'Elongate', 'Ovate', 'Round', 'Obovate', 'Lanceolate',
        'Kidney-shaped', 'Heart-shaped', 'Spathulate'
    ],
    'Petiole':
    [
        'Petiole', 'Sessile', 'Petiolated', 'Canaliculate', 'Glands', 'Glandular', 
     'Winged' 'Wings', 'Hairs', 'Hair', 'Translucent'
     ],
    'Leaf Colour':
    [
        'Leaf Colour', 'Leaf Color'
    ],
    'Leaf Blade':
    [
        'Leaf Blade', 'Linear', 'Lanceolate', 'Elliptical', 'Obovate', 'Obtriangular', 
        'Obtriangular', 'Asymmetrical', 'Orbicular', 'Bilobed', 'Lobed', 'Lobes', 'Lobe'
    ],
    'Leaf Base':
    [
        'Leaf Base', 'Rounded', 'Cordate', 'Glands'
    ],
    'Leaf Margin':
    [
        'Margin', 'Smooth', 'Wavy', 'Crenate', 'Toothed', 'Teeth', 'Crenate', 'Serrate'
    ],
    'Leaf Apex':
    [
        'Apex', 'Acuminate', 'Apiculate', 'Mucronate', 'Rounded', 'Emarginated'
    ],
    'Leaf side':
    [
        'Glabrous', 'Pubescent', 'Salt Crystals', 'Scales', 'Woolly', 'Powdery'
    ],
    'Leaf glands':
    [
        'Glands', 'Gland', 'Translucent'
    ],
    'Rachis':
    [
        'Rachis', 'Winged'
    ],
    'Vein':
    [
        'Vein'
    ],
    'Tendril':
    [
        'Tendril'
    ],
    'Spine':
    [
        'Spine', 'Prickle', 'Spines', 'Prickles'
    ],
    'Thornes':
    [
        'Thorn', 'Thornes'
    ],
    'Blade Colour':
    [
        'Blade'
    ],
    'Fruit':
    [
        'Drupe', 'Berry', 'Capsule', 'Pod', 'Follicle', 'Achene', 'Winged', 'Follicle',
        'Pod', 'Nutlet', 'Fruit'
    ],
    'Fruit Shape':
    [
        'locular', 'Globose', 'Flattened', 'Elongate', 'Obovoid', 'Ovate', 'Twisted',
        'Curved', 'Pyriform', 'Ovoid'
    ],
    'Fruit Colour':
    [
        'Fruit'
    ],
    'Inflorescences':
    [
        'Inflorescences', 'Inflorescence', 'Sessile', 'Panicle', 'Flower head', 'Cyme', 'Glomerule', 
        'Fascicle', 'Umbel', 'Corymb', 'Rootlet', 'Spike', 'Dichasium', 'Fascicle',
        'Globose', 'Raceme', 'Fascicle', 'Umbel'
     ],
    'Sexuality':
    [
        'Sexuality', 'Axillary', 'Terminal'
    ],
    'Flower Colour':
    [
        'Flower colour', 'Flower color', 'Flower', 'Flowers'
    ],
    'Flower Shape':
    [
        'Flower shape', 'Petalled', 'Petal', 'Petals', 'Tubular', 'Apetal', 'Butterfly-shaped', 'Shaped', 'Flower', 'Flowers'
    ],
    'Sepal Shape':
    [
        'Sepal', 'Sepals', 'Connate'
    ],
    'Petal Shape':
    [
        'Petal', 'Petals', 'Tepals', 'Tepal', 'Tubular'
    ],
    'Aril Colour':
    [
        'Aril'
    ],
    'Seed Colour':
    [
        'Seed', 
    ],
    'Conspicuousness':
    [
        'Conspicuousness', 'Cryptic'
    ]
}

with open('../../data/supportive/traits_Pierre_and_Andrei.json', 'w') as f:
    json.dump(traits_dict, f)
with open('../../data/supportive/colour_list.json', 'w') as f:
    json.dump(colors, f)

traits_list = list(traits_dict.keys())
traits_list += [trait.lower() for lst in list(traits_dict.values()) for trait in lst]

#### Functions

In [5]:
def corresponding_keys(val, dictionary):
    """returns the corresponding key of a single value 
    assuming the values are lists.

    Args:
        val (string): string present in the dict
        dictionary (dict): dict with lists of stings as values

    Returns:
        list: list of matching keys
    """
    # Init list
    keys = []
    # Search the dict
    for k, v in dictionary.items():
        if val in v:
            keys.append(k)
    return keys
    
def jaccard_similarity(A, B):
    """Calculates the Jaccard similarity two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    A = set(A)
    B = set(B)
    
    # Get intersection of two sets
    nominator = A.intersection(B)

    # Find union of two sets
    denominator = A.union(B)

    # Take the ratio of sizes
    similarity = len(nominator)/len(denominator)

    return similarity


def similarity(groundtruth, pred):
    """Calculates the normal similarity between two sets.

    Args:
        A (Set): Set A
        B (Set): Set B

    Returns:
        Integer: 0.00 - 1.00
    """

    # Create sets just in case
    groundtruth = set(groundtruth)
    pred = set(pred)

    # Find intersection of two sets
    nominator = groundtruth.intersection(pred)

    # Find union of two sets
    denominator = groundtruth

    # Take the ratio of sizes
    similarity = len(nominator)/(len(denominator))

    return similarity

def resentesize(lst):

    sentence = ''

    # Loop every 2 items 
    for item1, item2 in zip(lst[::2], lst[1::2]):
        
        # Set verb
        verb = 'has'
        chunk = item2.split(' ')[-1]
        if chunk not in nouns or chunk in colors:
            verb = 'is'

        sentence += f'{item1.capitalize()} {verb} {item2}. '
    
    return sentence


def get_wiki_main_image(title):
    url = 'https://en.wikipedia.org/w/api.php'
    data = {
        'action' :'query',
        'format' : 'json',
        'formatversion' : 2,
        'prop' : 'pageimages|pageterms',
        'piprop' : 'original',
        'titles' : title
    }
    response = requests.get(url, data)
    json_data = json.loads(response.text)
    return json_data['query']['pages'][0]['original']['source'] if len(json_data['query']['pages']) >0 else 'Not found'


### Load GT Data

In [6]:
root = "../../data/external/"

#### Data Andrei

In [7]:
file_name = root + 'Dataset_Andrei.csv'
df_Andrei = pd.read_csv(file_name)

# Get Dummies to match DF Pierre
df_Andrei_dummies = pd.get_dummies(df_Andrei.iloc[:, 2:])
# Set species back
df_Andrei_dummies = df_Andrei_dummies.set_index(df_Andrei['Species'])

# Create tuple list for multi index
Andrei_multi_index = []
for top_index in df_Andrei.columns:
    for sub_index in df_Andrei_dummies.columns:
        if top_index in sub_index:

            sub_index = sub_index.split('_')[-1]
            Andrei_multi_index.append((top_index, sub_index))

# Set Mutli index
df_Andrei_dummies.columns = pd.MultiIndex.from_tuples(Andrei_multi_index)
df_Andrei_dummies

Life form      Leaf position                      \
                               liana tree     alternate alternate, opposite   
Species                                                                       
Avicennia germinans                0    1             0                   0   
Metopium brownei                   0    1             1                   0   
Handroanthus billbergii            0    1             0                   0   
Bourreria succulenta               0    1             1                   0   
Bursera karsteniana                0    1             1                   0   
Bursera simaruba                   0    1             1                   0   
Bursera tomentosa                  0    1             1                   0   
Cynophalla flexuosa                1    0             1                   0   
Cynophalla hastata                 0    1             1                   0   
Quadrella indica                   0    1             1                   0   
Quadrella odoratissima             0    1             1                   0   
Crossopetalum rhacoma              0    1             0                   0   
Maytenus tetragona                 0    1             1                   0   
Maytenus versluysii                0    1             1                   0   
Clusia rosea                       0    1             0                   0   
Conocarpus erectus                 0    1             1                   0   
Laguncularia racemosa              0    1             0                   0   
Erythroxylum havanense             0    1             1                   0   
Croton niveus                      0    1             1                   0   
Hippomane mancinella               0    1             1                   0   
Libidibia coriaria                 0    1             1                   0   
Pithecellobium unguis-cati         0    1             1                   0   
Prosopis juliflora                 0    1             1                   0   
Vachellia tortuosa                 0    1             1                   0   
Vitex compressa                    0    1             0                   0   
Vitex cymosa                       0    1             0                   0   
Trichilia trifolia                 0    1             1                   0   
Eugenia procera                    0    1             0                   0   
Guapira pacurero                   0    1             0                   0   
Coccoloba swartzii                 0    1             1                   0   
Coccoloba uvifera                  0    1             1                   0   
Krugiodendron ferreum              0    1             0                   1   
Rhizophora mangle                  0    1             0                   0   
Guettarda roupalifolia             0    1             0                   0   
Randia aculeata                    0    1             0                   0   
Amyris ignea                       0    1             1                   0   
Zanthoxylum monophyllum            0    1             1                   0   
Casearia tremula                   0    1             1                   0   
Schoepfia schreberi                0    1             1                   0   
Jacquinia arborea                  0    1             1                   0   
Guaiacum officinale                0    1             0                   0   
Guaiacum sanctum                   0    1             0                   0   

                                                           \
                           opposite opposite, whorls of 3   
Species                                                     
Avicennia germinans               1                     0   
Metopium brownei                  0                     0   
Handroanthus billbergii           1                     0   
Bourreria succulenta              0                     0   
Bursera karsteniana               0         

#### Data Pierre

In [12]:
file_name = root + 'Dataset_Pierre.csv'
df_Pierre = pd.read_csv(file_name, header=[0, 1]) 
df_Pierre = df_Pierre.iloc[: , 1:]
df_Pierre = df_Pierre.set_index(df_Pierre['Species']['species'])
df_Pierre = df_Pierre.drop(columns=['Species', 'XXX'])
for i in df_Pierre:
    print(i)

('plant type', 'Tree')
('plant type', 'Shrub')
('plant type', 'Bush')
('plant type', 'Ficus strangler')
('plant type', 'Liana')
('plant type', 'Parasitic')
('plant type', 'Palm tree')
('plant type', 'Herbaceous')
('phyllotaxis', 'Phyllotaxis alternate')
('phyllotaxis', 'Opposite phyllotaxis')
('phyllotaxis', 'Phyllotaxis whorled')
('trunk and root', 'Base of trunk straight')
('trunk and root', 'Base of trunk flared')
('trunk and root', 'Foothills')
('trunk and root', 'Stilt roots')
('trunk and root', 'Aerial roots')
('latex', 'No exudate')
('latex', 'Clear latex')
('latex', 'Clear gum or resin')
('latex', 'White latex')
('latex', 'Yellow latex')
('latex', 'Dark gum or resin')
('crown', 'Regular density of the crown')
('crown', 'Leaves grouped at the top of the axes')
('crown', 'Horizontally spreading crowns')
('crown', 'drooping axles')
('crown', 'Stepped crown')
('crown', 'Candelabra-shaped crown')
('stem shape', 'Circular stem section')
('stem shape', 'Square stem section')
('bark', 

/var/folders/0p/7b4jnnp15d5gx16h36l4ycx00000gn/T/ipykernel_51047/3964626144.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_Pierre = df_Pierre.drop(columns=['Species', 'XXX'])


#### Data Palms

In [ ]:
file_name = root + 'Dataset_Kissling.txt'
df_Daniel = pd.read_csv(file_name,
                 sep='\t', encoding='Latin-1')
palm_species = df_Daniel[~df_Daniel.isnull().any(axis=1)]['SpecName'].values
df_Daniel.set_index('SpecName', inplace=True)

### RESHAPE DATA FOR PALMS

# Exclude string types
df_Daniels_int = df_Daniel.select_dtypes(exclude=[object])
df_Daniels_str = df_Daniel.select_dtypes(include=[object])
# Drop numbers
df_Daniels_semi_ints = df_Daniels_int.loc[:, df_Daniels_int.max() <= 3]
# Merge again
df_Daniel_edit = pd.merge(df_Daniels_str, df_Daniels_semi_ints, left_index=True, right_index=True)


# Real numbers:
df_Daniels_real_ints = df_Daniels_int.loc[:, df_Daniels_int.max() >= 3]
columns = [
    ("Measurement", "Maximum Stem Height"),
    ("Measurement", "Maximum Stem Diameter"),
    ("Measurement", "Maximum Leaf Number"),
    ("Measurement", "Maximum Leaf Blade Length"),
    ("Measurement", "Maximum Rachis Length"),
    ("Measurement", "Maximum Petiole Length"),
    ("Measurement", "Average Fruit Length"),
    ("Measurement", "Minimum Fruit Length"),
    ("Measurement", "Maximum Fruit Length"),
    ("Measurement", "Average Fruit Width"),
    ("Measurement", "Minimum Fruit Width"),
    ("Measurement", "Maximum Fruit Width"),
]

df_Daniels_real_ints.columns = pd.MultiIndex.from_tuples(columns)
df_Daniels_real_ints

# Get colors as lst of lsts 
FruitColorDescription_colors_lst = []

for palm_colors in df_Daniels_str['FruitColorDescription'].values:
    if type(palm_colors) == str:
        #print(type(colors))
        palm_colors = re.split(r'; |to | |-', palm_colors)

        #print(palm_colors)
        FruitColorDescription_colors_lst.append([color for color in palm_colors if color in colors])
    else:
        FruitColorDescription_colors_lst.append([])

MainFruitColors_colors_lst = []

for palm_colors in df_Daniels_str['MainFruitColors'].values:
    if type(palm_colors) == str:
        #print(type(colors))
        palm_colors = re.split(r'; |to | |-', palm_colors)

        #print(palm_colors)
        MainFruitColors_colors_lst.append([color for color in palm_colors if color in colors])
    else:
        MainFruitColors_colors_lst.append([])

# Init SKlearn MLB
mlb = MultiLabelBinarizer()

# Create dummies for color columns
df_FruitColorDescription = pd.DataFrame(
    {
        'FruitColorDescription': FruitColorDescription_colors_lst
    }, columns=['FruitColorDescription'])

s = df_FruitColorDescription['FruitColorDescription']
df_FruitColorDescription = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_Daniel.index)

# Multiindex columns
columns = [('Fruit Colour Description', column) for column in df_FruitColorDescription.columns]
df_FruitColorDescription.columns = pd.MultiIndex.from_tuples(columns)

# Create dummies for color columns
df_MainFruitColors = pd.DataFrame(
    {
        'MainFruitColors': MainFruitColors_colors_lst
    }, columns=['MainFruitColors'])

s = df_MainFruitColors['MainFruitColors']
df_MainFruitColors = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df_Daniel.index)

# Multiindex columns
columns = [('Fruit Colour', column) for column in df_MainFruitColors.columns]
df_MainFruitColors.columns = pd.MultiIndex.from_tuples(columns)

df_Daniels_str_non_color = df_Daniels_str[['UnderstoreyCanopy', 'FruitSizeCategorical', 'FruitShape', 'Conspicuousness']]

# df_Daniels_str_non_color.columns = pd.MultiIndex.from_tuples(
#     [
#         ('Crown', 'UnderstoreyCanopy'),
#         ('Fruit Size', 'FruitSizeCategorical'),
#         ('Fruit Shape', 'FruitShape'),
#         ('Conspicuousness', 'Conspicuousness'),
#     ]
# )

df_Daniels_str_non_color_dummies = pd.get_dummies(df_Daniels_str_non_color)
columns = []
for column in df_Daniels_str_non_color_dummies.columns:
    level0, level1 = column.split('_')
    if level0 == 'UnderstoreyCanopy':
        level0 = 'Crown'
    elif level0 == 'FruitSizeCategorical':
        level0 = 'Fruit Size'
    elif level0 == 'FruitShape':
        level0 = 'Fruit Shape'
    elif level0 == 'Conspicuousness':
        level0 = 'Conspicuousness'
    columns.append((level0, level1))
    
df_Daniels_str_non_color_dummies.columns = pd.MultiIndex.from_tuples(columns)

### JOIN ALL DATA
data_frames = [df_FruitColorDescription, df_MainFruitColors, df_Daniels_str_non_color_dummies, df_Daniels_real_ints]
df_Daniel_merged = pd.concat(data_frames, axis=1)
df_Daniel_merged

## Prediction Data

### Load Data

In [ ]:
root = "../../data/processed/"
sentences_all = {}

f = open(F"{root}Sentences_Pierre.pkl", 'rb')
sentences_Pierre = pickle.load(f)
sentences_all |= sentences_Pierre 

f = open(F"{root}Sentences_Andrei.pkl", 'rb')
sentences_Andrei = pickle.load(f)
sentences_all |= sentences_Andrei 

f = open(F"{root}Sentences_Kissling.pkl", 'rb')
sentences_Kissling = pickle.load(f)
sentences_all |= sentences_Kissling 

# Drop duplicates
for species, sentences in sentences_all.items():
    sentences_all[species] = list(set(sentences))


### Match Data

In [ ]:
k= 5
google_form_lst = []

for idx, (species, sentences) in enumerate(tqdm(sentences_all.items())):

    # if idx < 400:
    #     continue
    # if species != 'Dypsis thiryana':
    #     continue
    
    df_select = [df_Andrei_dummies, df_Pierre, df_Daniel_merged]
    if species in df_Andrei_dummies.index:
        df_select = df_select[0]
    elif species in df_Pierre.index:
        df_select = df_select[1]
    elif species in df_Daniel_merged.index:
        df_select = df_select[2]
    else:
        raise ValueError

    for gt_main_trait in df_select.columns.get_level_values(0).unique():
        if gt_main_trait == 'Measurement':
            df_subset = df_select[df_select.index == species][gt_main_trait]
            # Not really efficient, use PD?
    
            subtraits = list(df_subset.columns)
            values = df_subset.values[0]
            for sub_trait, value in zip(subtraits, values):

                df_sent  = F"{sub_trait} {value}"

                top_list = []

                for sentence in sentences:
                    gt_sim = jaccard_similarity(sentence, df_sent)
                    top_list.append((gt_sim, sentence))

                top_list.sort(reverse=True)
                top_k_list = [sentence for (_, sentence) in top_list[0:k]]

                google_form_lst.append((species, gt_main_trait, sub_trait, df_sent, *top_k_list))
        else:
            df_subset = df_select[df_select.index == species][gt_main_trait]
            present_traits = df_subset.loc[:, df_subset.any()].columns.values
            df_sent  = ' '.join(gt_main_trait + ' ' + present_traits)

            top_list = []

            for sentence in sentences:
                gt_sim = jaccard_similarity(sentence, df_sent)
                top_list.append((gt_sim, sentence))

            top_list.sort(reverse=True)
            top_k_list = [sentence for (_, sentence) in top_list[0:k]]

            google_form_lst.append((species, gt_main_trait, gt_main_trait, list(present_traits), *top_k_list))

### Create Subset

In [ ]:
# Google forms
df_google = pd.DataFrame(google_form_lst, columns=['Species', 'Main Trait', 'GT Main Trait', 'GT Sub Traits', '1', '2', '3', '4', '5'])
df_google

In [ ]:
df_google_subset = df_google\
                    .dropna()\
                    .sample(n=20, axis=0, random_state=666)\

df_google_subset

### Add Images

In [ ]:
random_sample_species = df_google_subset['Species'].values

images_links = {}
for species in tqdm(random_sample_species):
    species_ = species.replace(' ', '_')
    try:
        img_url = get_wiki_main_image(species_)
        if img_url[-3:] != 'jpg':
            img_url = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'
        # if img_url[-3:].isupper():
        #     img_url = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'
        images_links[species] = img_url
    except:
        images_links[species] = 'https://upload.wikimedia.org/wikipedia/commons/1/14/No_Image_Available.jpg'

# df_google_subset = df_google[df_google['Species'].isin(random_sample_species)]
df_google_subset = df_google_subset.set_index('Species')
df_google_subset["URL"] = pd.Series(images_links)
df_google_subset.to_csv(f'{root}top_sents_all_AllSentencesAgainstTrait_Random20Subset.csv', sep='\t')#, quoting=csv.QUOTE_ALL)

In [ ]:
df_google_subset